# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [56]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [57]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv('output_data/cities.csv')

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tiksi,71.6872,128.8694,-20.47,100,58,1.73,RU,1698695908
1,1,adamstown,-25.0660,-130.1015,22.71,75,100,4.26,PN,1698695908
2,2,hawaiian paradise park,19.5933,-154.9731,27.16,92,100,2.57,US,1698695616
3,3,keflavik,64.0049,-22.5624,3.24,75,40,6.17,IS,1698695922
4,4,port-aux-francais,-49.3500,70.2167,2.91,80,100,15.76,TF,1698695924


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [58]:
# %%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    size = 'Humidity',
    color = 'City'
)

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [59]:
# Narrow down cities that fit criteria and drop any results with null values
condition = city_data_df[(city_data_df.loc[:, 'Cloudiness'] == 0)]
                         
# Drop any rows with null values
city_data_clean = condition.dropna(how = 'any')

# Display sample data
city_data_clean

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,port elizabeth,-33.9180,25.5701,16.16,77,0,8.23,ZA,1698695952
21,21,haiku-pauwela,20.9219,-156.3051,27.80,71,0,4.12,US,1698695959
29,29,puerto natales,-51.7236,-72.4875,7.25,39,0,1.54,CL,1698695968
39,39,malabar,-33.9667,151.2500,26.84,21,0,8.23,AU,1698696023
43,43,fort bragg,35.1390,-79.0060,30.08,44,0,7.72,US,1698695930
...,...,...,...,...,...,...,...,...,...,...
488,488,gushu,31.5598,118.4829,11.99,72,0,1.90,CN,1698696293
495,495,kruisfontein,-34.0033,24.7314,12.89,66,0,6.65,ZA,1698696295
513,513,qionghu,28.8430,112.3522,18.91,63,0,1.10,CN,1698696299
522,522,ankazoabo,-22.2833,44.5167,22.27,66,0,6.37,MG,1698696302


### Step 3: Create a new DataFrame called `hotel_df`.

In [60]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_clean.copy().drop(columns = ['City_ID', 'Max Temp', 'Cloudiness', 'Wind Speed', 'Date'])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(len(hotel_df.columns), 'Hotel Name', value = '')

# Display sample data
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
10,port elizabeth,-33.9180,25.5701,77,ZA,
21,haiku-pauwela,20.9219,-156.3051,71,US,
29,puerto natales,-51.7236,-72.4875,39,CL,
39,malabar,-33.9667,151.2500,21,AU,
43,fort bragg,35.1390,-79.0060,44,US,
...,...,...,...,...,...,...
488,gushu,31.5598,118.4829,72,CN,
495,kruisfontein,-34.0033,24.7314,66,ZA,
513,qionghu,28.8430,112.3522,63,CN,
522,ankazoabo,-22.2833,44.5167,66,MG,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [62]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'limit' : 1,
    'filter' : '',
    'bias' : '',
    'categories' : 'accommodation.hotel',
    'apiKey' : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, 'Lat']
    longitude = hotel_df.loc[index, 'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port elizabeth - nearest hotel: Waterford Hotel
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
puerto natales - nearest hotel: Dorotea Patagonia Hostel
malabar - nearest hotel: Coogee Bay Boutique Hotel
fort bragg - nearest hotel: Airborne Inn Lodging
kristiansund - nearest hotel: Comfort Hotel Fosna
menaka - nearest hotel: No hotel found
pahrump - nearest hotel: Holiday Inn Express & Suites
vila velha - nearest hotel: Hotel Prainha
zakynthos - nearest hotel: Phoenix Hotel
arraial do cabo - nearest hotel: No hotel found
ulsteinvik - nearest hotel: Quality Hotel Ulstein
at taj - nearest hotel: No hotel found
dwarka - nearest hotel: The Dwarika Hotel
laguna - nearest hotel: Holiday Inn Express & Suites
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
kilju - nearest hotel: No hotel found
sosnovo-ozerskoye - nearest hotel: Озёрная
el granada - nearest hotel: Beach House
zuni pueblo - nearest hotel: No hotel found
carnarvon - nearest hote

,City,Lat,Lng,Humidity,Country,Hotel Name
10,port elizabeth,-33.9180,25.5701,77,ZA,Waterford Hotel
21,haiku-pauwela,20.9219,-156.3051,71,US,Inn At Mama's Fish House
29,puerto natales,-51.7236,-72.4875,39,CL,Dorotea Patagonia Hostel
39,malabar,-33.9667,151.2500,21,AU,Coogee Bay Boutique Hotel
43,fort bragg,35.1390,-79.0060,44,US,Airborne Inn Lodging
...,...,...,...,...,...,...
488,gushu,31.5598,118.4829,72,CN,No hotel found
495,kruisfontein,-34.0033,24.7314,66,ZA,No hotel found
513,qionghu,28.8430,112.3522,63,CN,No hotel found
522,ankazoabo,-22.2833,44.5167,66,MG,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [63]:
#%%capture --no-display

# Configure the map plot
hotel_map = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    size = 'Humidity',
    color = 'City',
    hotel = ['Hotel Name', 'Country']
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)